# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [28]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [29]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df['col2']=[6,7,8,9]
df

col1  col2
0     1     6
1     2     7
2     3     8
3     4     9

## 2. Deleting a row in a DataFrame

In [30]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df.drop('d',axis=0)

col1
a     1
b     2
c     3

## 3. Creating a DataFrame from a few Series

In [31]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))
df=pd.DataFrame({'col1':ser_1,'col2':ser_2,'col3':ser_3})
df

col1      col2      col3
0 -1.832920 -2.134158 -1.987818
1 -0.456820 -1.054451 -0.502023
2  0.084541 -1.016252  0.940325
3  1.257276 -0.989799 -1.107502
4 -1.250072 -0.265249  1.243225
5 -0.788423  0.166057 -1.010534

# Pandas questions: Indexing

## 1. Indexing into a specific column

In [32]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df['col_2']

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [33]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.loc['obs3']

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [34]:
# using the same DataFrame, index into into its first row
df.iloc[0]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [35]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'],engine='python')

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'],engine='python')

## 2. How to load the training and testing subsets

In [36]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('C:/Users/Sathvik/Desktop/DS/Recommendation engine/1489104021_pycon2015_tutorial322/pycon2015_tutorial322/data/movielens_train.csv', index_col=0,encoding='latin-1')
movielens_test = pd.read_csv('C:/Users/Sathvik/Desktop/DS/Recommendation engine/1489104021_pycon2015_tutorial322/pycon2015_tutorial322/data/movielens_test.csv', index_col=0,encoding='latin-1')

In [37]:
movielens_train.head()

user_id  movie_id  rating  timestamp gender  age  occupation    zip  \
593263     3562      3798       4  967332344      F   25           6  32812   
235597     1051      3793       4  974958593      F   25           0  60513   
219003     3727      2366       3  966309522      M   35           7  74401   
685090     4666      1094       3  963843918      M   35           1  53704   
312377     3261      1095       4  968251750      M   45          20  87505   

                             title                   genres for_testing  
593263    What Lies Beneath (2000)                 Thriller       False  
235597                X-Men (2000)            Action|Sci-Fi       False  
219003            King Kong (1933)  Action|Adventure|Horror       False  
685090     Crying Game, The (1992)        Drama|Romance|War       False  
312377  Glengarry Glen Ross (1992)                    Drama       False

In [38]:
movielens_test.head()

user_id  movie_id  rating   timestamp gender  age  occupation    zip  \
693323     4653      2648       4   975532459      M   35          12  95051   
24177      2259      1270       4   974591524      F   56          16  70503   
202202     3032      1378       5   970343147      M   25           0  47303   
262003     3029      2289       4   972846393      M   18           4  92037   
777848     4186      2403       3  1017931262      M   25           7  33308   

                            title                 genres for_testing  
693323        Frankenstein (1931)                 Horror       False  
24177   Back to the Future (1985)          Comedy|Sci-Fi       False  
202202          Young Guns (1988)  Action|Comedy|Western       False  
262003         Player, The (1992)           Comedy|Drama       False  
777848         First Blood (1982)                 Action       False

# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [39]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [40]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [41]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [42]:
print('RMSE for my estimate function: %s' % evaluate(my_estimate_func))


RMSE for my estimate function: 1.23237195265


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [44]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    user_condition= movielens_train.user_id!=user_id #not the same user
    movie_condition= movielens_train.movie_id == movie_id
    ratings_by_other_users=movielens_train.loc[user_condition & movie_condition]
    
    if ratings_by_other_users.empty:
        return 3.0
    else:
        return ratings_by_other_users.rating.mean()
    # first, index into all ratings of this movie
    # second, compute the mean of those ratings
    # 

    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

4.0

# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities

# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [72]:
def collab_age(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on age. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means_by_age = ratings_by_others.pivot_table('rating', index='movie_id', columns='age')
    user_age = user_info.ix[user_id, 'age']
    if user_age in means_by_age.columns: 
        return means_by_age.ix[movie_id, user_age]
    else:
        return means_by_age.ix[movie_id].mean()

print('RMSE for collab_gender: %s' % evaluate(collab_age))

RMSE for collab_gender: 1.20520133441


In [73]:
def collab_zip(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on zip. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means_by_zip = ratings_by_others.pivot_table('rating', index='movie_id', columns='zip')
    user_zip = user_info.ix[user_id, 'zip']
    if user_zip in means_by_zip.columns: 
        return means_by_zip.ix[movie_id, user_zip]
    else:
        return means_by_zip.ix[movie_id].mean()

print('RMSE for collab_gender: %s' % evaluate(collab_zip))

RMSE for collab_gender: 1.12566403192


In [74]:
def collab_occupation(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on occupation. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means_by_occupation = ratings_by_others.pivot_table('rating', index='movie_id', columns='occupation')
    user_occupation = user_info.ix[user_id, 'occupation']
    if user_occupation in means_by_occupation.columns: 
        return means_by_occupation.ix[movie_id, user_occupation]
    else:
        return means_by_occupation.ix[movie_id].mean()

print('RMSE for collab_gender: %s' % evaluate(collab_occupation))

RMSE for collab_gender: 1.20287696436


In [75]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

In [76]:
def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [77]:
class CollabPearsonReco:
    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')
        
    def estimate(self, user_id, movie_id):
    
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: euclidean(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)

reco = CollabPearsonReco()
reco.learn()
print('RMSE for CollabPearsonReco: %s' % evaluate(reco.estimate))

RMSE for CollabPearsonReco: 1.123429476


The RMSE for cosine similarity was 1.13439921696 while the RMSE for euclidean similarity is 1.123429476